In [7]:
import sys  # System-specific parameters and functions
import subprocess  # Spawn new processes, connect to their input/output/error pipes, and obtain their return codes
import os  # Operating system dependent functionality
import gc  # Garbage Collector interface
from pathlib import Path  # Object-oriented filesystem paths
from glob import glob  # Unix style pathname pattern expansion

import numpy as np  # Fundamental package for scientific computing with Python
import pandas as pd  # Powerful data structures for data manipulation and analysis
import polars as pl  # Fast DataFrame library implemented in Rust

from datetime import datetime  # Basic date and time types
import seaborn as sns  # Statistical data visualization
import matplotlib.pyplot as plt  # MATLAB-like plotting framework

import joblib  # Save and load Python objects

import warnings  # Warning controlj
warnings.filterwarnings('ignore')  # Ignore warnings

from sklearn.base import BaseEstimator, RegressorMixin  # Base classes for all estimators in scikit-learn
from sklearn.metrics import roc_auc_score  # ROC AUC score
import lightgbm as lgb  # LightGBM: Gradient boosting framework
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold  # Cross-validation strategies
from imblearn.over_sampling import SMOTE  # Oversampling technique for imbalanced datasets
from sklearn.preprocessing import OrdinalEncoder  # Encode categorical features as an integer array
from sklearn.impute import KNNImputer  # Imputation for completing missing values using k-Nearest Neighbors



In [8]:
ROOT = '/kaggle/input/home-credit-credit-risk-model-stability'  # Setting the root directory path


# 🛠️📊 Pipeline for  Data Preprocessing 

Let's create a  class named `Pipeline` containing methods to preprocess data using Pandas and Pipelines. 
**1. `set_table_dtypes(df)`**
- This method iterates through each column in the DataFrame (`df`) and converts the data types based on certain conditions.
- If the column name is one of ["case_id", "WEEK_NUM", "num_group1", "num_group2"], it converts the column to `Int64`.
- If the column name is "date_decision", it converts the column to `Date`.
- If the last character of the column name is "P" or "A", it converts the column to `Float64`.
- If the last character of the column name is "M", it converts the column to `String`.
- If the last character of the column name is "D", it converts the column to `Date`.
- Finally, it returns the DataFrame with modified data types.

**2. `handle_dates(df)`**
- This method aims to handle date columns in the DataFrame.
- It iterates through each column, and if the last character of the column name is "D", it performs some operations.
- It subtracts the date values in the current column from the values in the "date_decision" column.
- Then it computes the total days between the two dates.
- After processing, it drops the "date_decision" and "MONTH" columns from the DataFrame.
- Finally, it returns the modified DataFrame.

**3. `filter_cols(df)`**
- This method filters out columns based on certain conditions.
- It iterates through each column and checks if the column name is not in ["target", "case_id", "WEEK_NUM"] and if the column type is `String`.
- If the number of unique values in the column is either 1 or more than 200, it drops that column.
- Finally, it returns the filtered DataFrame.

### Study Sources
- For learning Pandas and data preprocessing: [Pandas Documentation](https://pandas.pydata.org/docs/)
- Understanding Pipelines in data preprocessing: [Scikit-Learn Pipeline Documentation](https://scikit-learn.org/stable/modules/compose.html#pipeline)
- Data type conversion and manipulation: [Pandas Data Types and Conversion](https://pandas.pydata.org/pandas-docs/version/1.3/user_guide/basics.html#basics-dtypes)

In [9]:

class Pipeline:

    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))  #!!?
                df = df.with_columns(pl.col(col).dt.total_days()) # t - t-1
        df = df.drop("date_decision", "MONTH")
        return df

    def filter_cols(df):
        
        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 200):
                    df = df.drop(col)
        
        return df




# 🔍 Aggregator for Feature Extraction 

Let's create a  `Aggregator` class  to aggregate features from a DataFrame. 

**1. `num_expr(df)`**
- This method extracts numerical features from the DataFrame (`df`).
- It selects columns whose names end with "P" or "A", indicating some numerical measurements.
- For each selected column, it creates an expression to compute the maximum value and aliases it accordingly.
- Finally, it returns a list of expressions for maximum values of numerical features.

 **2. `date_expr(df)`**
- This method extracts date-related features from the DataFrame (`df`).
- It selects columns whose names end with "D", representing date columns.
- Similar to `num_expr`, it creates expressions to compute the maximum date value for each selected column and aliases them.
- It returns a list of expressions for maximum date values of date features.

 **3. `str_expr(df)`**
- This method extracts string features from the DataFrame (`df`).
- It selects columns whose names end with "M", indicating string type columns.
- It creates expressions to compute the maximum string value for each selected column and aliases them accordingly.
- Returns a list of expressions for maximum string values of string features.

 **4. `other_expr(df)`**
- This method extracts other miscellaneous features from the DataFrame (`df`).
- It selects columns whose names end with "T" or "L".
- Similar to previous methods, it computes the maximum value for each selected column and aliases them.
- Returns a list of expressions for maximum values of miscellaneous features.

 **5. `count_expr(df)`**
- This method extracts count-related features from the DataFrame (`df`).
- It selects columns containing "num_group" in their names.
- It computes the maximum value for each selected column and aliases them.
- Returns a list of expressions for maximum count values of count features.

**6. `get_exprs(df)`**
- This method aggregates all the expressions from the previous methods to get a comprehensive list of feature extraction expressions.
- It calls all the individual feature extraction methods and concatenates the resulting lists.
- Returns a consolidated list of expressions for all types of features.

### Study Sources
- For learning about feature extraction and aggregation: [Feature Engineering for Machine Learning](https://www.amazon.com/Feature-Engineering-Machine-Learning-Principles/dp/1491953241)
- Understanding Pandas DataFrame manipulation: [Pandas Documentation](https://pandas.pydata.org/docs/)
- Relational Algebra and Expressions: [Relational Algebra - Wikipedia](https://en.wikipedia.org/wiki/Relational_algebra)

In [10]:
class Aggregator:
    # Please add or subtract features yourself, be aware that too many features will take up too much space.
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        # expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        expr_median = [pl.median(col).alias(f"median_{col}") for col in cols]
        expr_var = [pl.var(col).alias(f"var_{col}") for col in cols]

        return expr_max + expr_last + expr_mean 

    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        # expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        # expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        expr_median = [pl.median(col).alias(f"median_{col}") for col in cols]

        return expr_max + expr_last + expr_mean 

    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        # expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        # expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        # expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]
        return expr_max + expr_last  # +expr_count

    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        # expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        # expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return expr_max + expr_last

    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        # expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        # expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return expr_max + expr_last

    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

# File Reading with Data Preprocessing 📄

The function `read_file(path, depth=None)` reads a Parquet file located at the given `path`, performs data preprocessing using the `Pipeline` class, and optionally aggregates features based on the `depth` parameter using the `Aggregator` class. 

1. `read_file(path, depth=None)`
- **Inputs**:
  - `path`: Path to the Parquet file.
  - `depth`: An optional parameter indicating the depth of feature aggregation. Default is `None`.
- **Output**: Returns a processed DataFrame.
- **Process**:
  - Reads the Parquet file located at the given `path` using `pl.read_parquet(path)`.
  - Performs data preprocessing using the `Pipeline` class by applying the `set_table_dtypes` method to ensure proper data types.
  - If `depth` is provided and is either 1 or 2:
    - It groups the DataFrame by "case_id".
    - It aggregates features based on the depth using the `Aggregator` class and the `get_exprs` method.
  - Returns the processed DataFrame.

### Study Sources
- For understanding Parquet file format and reading: [Parquet File Format](https://parquet.apache.org/documentation/latest/)
- Data preprocessing with Pandas Pipelines: [Pandas Pipe Documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html)
- Feature aggregation and group-by operations: [Pandas GroupBy Documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)
- Aggregating features using Pandas: [Pandas Aggregation Documentation](https://pandas.pydata.org/docs/user_guide/groupby.html#aggregation)

In [11]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    if depth in [1,2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df)) 
    return df



# 📄Reading Multiple Files with Data Preprocessing 

Let's create a function `read_files(regex_path, depth=None)` that reads multiple Parquet files matching the specified regex pattern, performs data preprocessing using the `Pipeline` class, optionally aggregates features based on the `depth` parameter using the `Aggregator` class, and concatenates the results.
**1. `read_files(regex_path, depth=None)`**
- **Inputs**:
  - `regex_path`: Regular expression pattern for matching file paths.
  - `depth`: An optional parameter indicating the depth of feature aggregation. Default is `None`.
- **Output**: Returns a concatenated and processed DataFrame.
- **Process**:
  - Initializes an empty list `chunks` to store processed DataFrames.
  - Iterates through each file path matched by the provided regular expression pattern using `glob(str(regex_path))`.
    - Reads each Parquet file using `pl.read_parquet(path)`.
    - Performs data preprocessing using the `Pipeline` class by applying the `set_table_dtypes` method.
    - If `depth` is provided and is either 1 or 2:
      - It groups the DataFrame by "case_id".
      - It aggregates features based on the depth using the `Aggregator` class and the `get_exprs` method.
    - Appends the processed DataFrame to the `chunks` list.
  - Concatenates all DataFrames in `chunks` vertically using `pl.concat(chunks, how="vertical_relaxed")`.
  - Removes duplicate rows based on the "case_id" column using `df.unique(subset=["case_id"])`.
  - Returns the concatenated and processed DataFrame.

### Study Sources
- For understanding file path manipulation and regular expressions: [Python Glob Documentation](https://docs.python.org/3/library/glob.html)
- Concatenating DataFrames in Pandas: [Pandas Concatenation Documentation](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
- Removing duplicate rows in Pandas: [Pandas Drop Duplicates Documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [12]:
def read_files(regex_path, depth=None):
    chunks = []
    
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)
    
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    return df



# 🛠️ Feature Engineering Function 

Function `feature_eng(df_base, depth_0, depth_1, depth_2)` performs feature engineering on a base DataFrame (`df_base`) and multiple sets of additional DataFrames (`depth_0`, `depth_1`, `depth_2`). It adds new features, joins additional DataFrames, and handles dates using the `Pipeline` class.

 **1. `feature_eng(df_base, depth_0, depth_1, depth_2)`**
- **Inputs**:
  - `df_base`: Base DataFrame on which feature engineering will be performed.
  - `depth_0`, `depth_1`, `depth_2`: Lists of DataFrames representing additional features of different depths.
- **Output**: Returns the feature-engineered DataFrame.
- **Process**:
  - Adds new features to the base DataFrame:
    - `month_decision`: Extracts the month from the "date_decision" column.
    - `weekday_decision`: Extracts the weekday from the "date_decision" column.
  - Iterates through each set of additional DataFrames (`depth_0`, `depth_1`, `depth_2`):
    - Joins each DataFrame to the base DataFrame using the "case_id" column as the key and left join method.
    - Appends a suffix to the column names to distinguish between different sets of features.
  - Performs date handling using the `Pipeline` class by applying the `handle_dates` method.
  - Returns the feature-engineered DataFrame.

### Study Sources
- For understanding feature engineering techniques: [Feature Engineering for Machine Learning](https://www.amazon.com/Feature-Engineering-Machine-Learning-Principles/dp/1491953241)
- Handling dates in Pandas: [Pandas DateTime Documentation](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)
- Joining DataFrames in Pandas: [Pandas Merge Documentation](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)
- Applying functions to Pandas DataFrame using pipe: [Pandas Pipe Documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html)

In [13]:
def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base



# 🐼 DataFrame Conversion to Pandas with Categorical Columns 

The function `to_pandas(df_data, cat_cols=None)` converts a DataFrame (`df_data`) to a Pandas DataFrame and optionally converts specified columns to categorical data type. 

 **1. `to_pandas(df_data, cat_cols=None)`**
- **Inputs**:
  - `df_data`: Input DataFrame to be converted to Pandas.
  - `cat_cols`: Optional list of column names to be converted to categorical data type. Default is `None`.
- **Output**: Returns the converted Pandas DataFrame and the list of categorical column names.
- **Process**:
  - Converts the input DataFrame to Pandas DataFrame using the `.to_pandas()` method.
  - If `cat_cols` is not provided, it selects columns with data type "object" as default categorical columns.
  - Converts the selected categorical columns to the categorical data type using `.astype("category")`.
  - Returns the converted Pandas DataFrame along with the list of categorical column names.

### Study Sources
- Converting Dask DataFrame to Pandas: [Dask DataFrame to Pandas](https://docs.dask.org/en/latest/dataframe-best-practices.html#converting-to-pandas)
- Converting column data types in Pandas: [Pandas DataFrame astype](https://pandas.pydata.org/pandas-docs/version/1.3/reference/api/pandas.DataFrame.astype.html)
- Handling categorical data in Pandas: [Categorical Data in Pandas](https://pandas.pydata.org/pandas-docs/version/1.3/user_guide/categorical.html)

In [14]:
def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data, cat_cols



# 🔽 Memory Reduction Function for DataFrames 

The function `reduce_mem_usage(df)` iterates through all columns of a DataFrame and modifies the data types to reduce memory usage. 

 **1. `reduce_mem_usage(df)`**
- **Input**: 
  - `df`: Input DataFrame.
- **Output**: Returns the DataFrame with reduced memory usage.
- **Process**:
  - Calculates the initial memory usage of the DataFrame (`start_mem`) using `df.memory_usage()`.
  - Iterates through each column of the DataFrame:
    - Checks if the column type is a category. If so, skips to the next column.
    - For non-category columns:
      - Determines the minimum and maximum values of the column (`c_min` and `c_max`).
      - If the column type is integer:
        - Checks if the data can be fit into `int8`, `int16`, `int32`, or `int64` and converts the column type accordingly.
      - If the column type is float:
        - Checks if the data can be fit into `float16`, `float32`, or `float64` and converts the column type accordingly.
      - If the column type is object (string), it skips the conversion.
  - Calculates the final memory usage of the DataFrame (`end_mem`) after the modifications.
- **Returns** the DataFrame with reduced memory usage.

### Study Sources
- Optimizing memory usage in Pandas: [Optimizing Memory Usage in Pandas](https://www.dataquest.io/blog/pandas-big-data/)
- Understanding data types and memory in Pandas: [Pandas Data Types and Memory Usage](https://pbpython.com/pandas_dtypes.html)
- Data type conversion in NumPy: [NumPy Data Types](https://numpy.org/doc/stable/reference/arrays.scalars.html#arrays-scalars-built-in)

In [15]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="category":
            continue
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2    
    return df


 **Definition of Root Directory and Subdirectories**
- `ROOT`: It specifies the root directory path where the dataset is located. The `Path` object is created using the `Path` class from the `pathlib` module.
- `TRAIN_DIR`: It specifies the directory path for training data files. It is derived from the `ROOT` directory by appending the subdirectories "parquet_files" and "train" using the `/` operator.
- `TEST_DIR`: It specifies the directory path for test data files. Similar to `TRAIN_DIR`, it is derived from the `ROOT` directory by appending the subdirectories "parquet_files" and "test" using the `/` operator.

### Study Sources
- Working with file paths in Python: [Pathlib Documentation](https://docs.python.org/3/library/pathlib.html)
- Manipulating file paths using Pathlib: [Pathlib Tutorial](https://realpython.com/python-pathlib/)

In [16]:
ROOT            = Path("/kaggle/input/home-credit-credit-risk-model-stability")

TRAIN_DIR       = ROOT / "parquet_files" / "train"



**Explaination** 


Initializes a dictionary `data_store` containing different sets of DataFrames obtained from reading Parquet files using the `read_file()` and `read_files()` functions.

****1. Data Store Initialization**
- `data_store`: It is a dictionary storing different sets of DataFrames under different keys.
  
**2. Data Read Operations**
- `df_base`: It stores the DataFrame obtained by reading the file "train_base.parquet" located in the `TRAIN_DIR` directory using the `read_file()` function.
- `depth_0`: It stores a list of DataFrames obtained by reading multiple files. The first element is obtained using the `read_file()` function, while the second element is obtained using the `read_files()` function with a wildcard pattern.
- `depth_1`: It stores a list of DataFrames obtained by reading multiple files using the `read_files()` function with specific patterns. Each file is associated with a depth level of 1.
- `depth_2`: It stores a list containing a single DataFrame obtained by reading a specific file associated with a depth level of 2 using the `read_file()` function.

### Study Sources
- Loading data from Parquet files: [Dask Parquet Reader](https://docs.dask.org/en/latest/dataframe-io.html#dask.dataframe.read_parquet)
- Working with dictionaries in Python: [Python Dictionaries](https://realpython.com/python-dicts/)

In [17]:

data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
    ]
}



In [18]:
data_store

{'df_base': shape: (1_526_659, 5)
 ┌─────────┬───────────────┬────────┬──────────┬────────┐
 │ case_id ┆ date_decision ┆ MONTH  ┆ WEEK_NUM ┆ target │
 │ ---     ┆ ---           ┆ ---    ┆ ---      ┆ ---    │
 │ i64     ┆ date          ┆ i64    ┆ i64      ┆ i64    │
 ╞═════════╪═══════════════╪════════╪══════════╪════════╡
 │ 0       ┆ 2019-01-03    ┆ 201901 ┆ 0        ┆ 0      │
 │ 1       ┆ 2019-01-03    ┆ 201901 ┆ 0        ┆ 0      │
 │ 2       ┆ 2019-01-04    ┆ 201901 ┆ 0        ┆ 0      │
 │ 3       ┆ 2019-01-03    ┆ 201901 ┆ 0        ┆ 0      │
 │ 4       ┆ 2019-01-04    ┆ 201901 ┆ 0        ┆ 1      │
 │ …       ┆ …             ┆ …      ┆ …        ┆ …      │
 │ 2703450 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
 │ 2703451 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
 │ 2703452 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
 │ 2703453 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
 │ 2703454 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
 └─────────┴───────────────┴────────┴─

#  Data Preprocessing and Feature Engineering 🔧

Perform data preprocessing and feature engineering operations on the training data.

**1. Data Preprocessing and Feature Engineering**
- `df_train = feature_eng(**data_store)`: Applies feature engineering to the training data stored in the `data_store` dictionary using the `feature_eng` function. The unpacking operator `**` is used to pass the dictionary as keyword arguments.
- `del data_store`: Deletes the `data_store` dictionary to release memory.
- `gc.collect()`: Manually triggers garbage collection to free up memory space.

 **2. Data Filtering, Conversion, and Memory Reduction**
- `df_train = df_train.pipe(Pipeline.filter_cols)`: Applies column filtering using the `filter_cols` method from the `Pipeline` class to the `df_train` DataFrame using the `pipe` method.
- `df_train, cat_cols = to_pandas(df_train)`: Converts the `df_train` DataFrame to Pandas DataFrame and retrieves the categorical column names. It uses the `to_pandas` function for conversion.
- `df_train = reduce_mem_usage(df_train)`: Reduces memory usage of the `df_train` DataFrame using the `reduce_mem_usage` function to optimize memory consumption.

 **3. Handling Missing Values**
- `nums = df_train.select_dtypes(exclude='category').columns`: Selects numerical columns (excluding categorical columns) from the DataFrame and stores their column names in the `nums` variable.
- `from itertools import combinations, permutations`: Imports the `combinations` and `permutations` functions from the `itertools` module.
- `nans_df = df_train[nums].isna()`: Creates a DataFrame `nans_df` to identify missing values in numerical columns.
- `nans_groups = {}`: Initializes an empty dictionary to store numerical columns grouped by the count of missing values.
- Loops through each numerical column (`col`) and calculates the count of missing values for each column. Then, it groups the columns based on the count of missing values in the `nans_groups` dictionary.

 **4. Memory Management**
- `del nans_df`: Deletes the `nans_df` DataFrame to release memory.
- `x = gc.collect()`: Manually triggers garbage collection to free up memory space.

### Study Sources
- Data preprocessing techniques: [Data Preprocessing with Pandas](https://realpython.com/python-data-preprocessing/)
- Feature engineering concepts: [Feature Engineering for Machine Learning](https://www.amazon.com/Feature-Engineering-Machine-Learning-Principles/dp/1491953241)
- Memory management in Python: [Memory Management in Python](https://realpython.com/python-memory-management/)
- Missing data handling: [Handling Missing Data with Pandas](https://pandas.pydata.org/pandas-docs/version/1.3/user_guide/missing_data.html)

In [19]:
df_train = feature_eng(**data_store)
del data_store
gc.collect()
df_train = df_train.pipe(Pipeline.filter_cols)
df_train, cat_cols = to_pandas(df_train)
df_train = reduce_mem_usage(df_train)
nums=df_train.select_dtypes(exclude='category').columns
from itertools import combinations, permutations
nans_df = df_train[nums].isna()
nans_groups={}
for col in nums:
    cur_group = nans_df[col].sum()
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]
del nans_df; x=gc.collect()



In [20]:
df_train

,case_id,WEEK_NUM,target,month_decision,weekday_decision,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,...,last_pmts_pmtsoverdue_635A,mean_pmts_dpdvalue_108P,mean_pmts_pmtsoverdue_635A,max_pmts_date_1107D,last_pmts_date_1107D,mean_pmts_date_1107D,max_num_group1_12,max_num_group2,last_num_group1_12,last_num_group2
0,0,0,0,1,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,1,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,0,1,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,0,1,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,1,1,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,2703450,91,0,10,1,NaN,NaN,-998.0,NaN,52863.589844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526655,2703451,91,0,10,1,NaN,NaN,-5592.0,NaN,324608.531250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526656,2703452,91,0,10,1,NaN,NaN,NaN,NaN,102738.757812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526657,2703453,91,0,10,1,NaN,NaN,-4616.0,NaN,212683.296875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Explaination**

Function `reduce_group(grps)` aims to reduce the number of columns within each group by selecting the column with the highest number of unique values. 

**1. `reduce_group(grps)`**
- **Input**:
  - `grps`: List of groups, where each group is represented as a list of column names.
- **Output**: Returns a list of selected columns within each group.
- **Process**:
  - Initializes an empty list `use` to store the selected columns within each group.
  - Iterates through each group `g` in the input list `grps`.
    - Initializes variables `mx` and `vx` to track the maximum number of unique values and the corresponding column name within the group, respectively.
    - Iterates through each column `gg` in the group `g`.
      - Calculates the number of unique values `n` in the column `df_train[gg]`.
      - Updates `mx` and `vx` if `n` is greater than the current maximum number of unique values.
    - Appends the column name `vx` with the highest number of unique values to the `use` list for the current group.
- **Returns** the list `use` containing selected columns within each group.

### Study Sources
- Working with groups and group operations: [Pandas GroupBy Documentation](https://pandas.pydata.org/pandas-docs/version/1.3/reference/groupby.html)
- Unique values in Pandas Series: [Pandas nunique Documentation](https://pandas.pydata.org/pandas-docs/version/1.3/reference/api/pandas.Series.nunique.html)
- Iterating through lists in Python: [Python List Iteration](https://realpython.com/iterate-through-dictionary-python/)

In [21]:
def reduce_group(grps):
    use = []
    for g in grps:
        mx = 0; vx = g[0]
        for gg in g:
            n = df_train[gg].nunique()
            if n>mx:
                mx = n
                vx = gg
        use.append(vx)
    return use



**Explaination** 

Function, `group_columns_by_correlation(matrix, threshold=0.8)`, aims to group columns based on their correlation values. 

 **1. `group_columns_by_correlation(matrix, threshold=0.8)`**
- **Inputs**:
  - `matrix`: DataFrame representing the dataset.
  - `threshold`: Threshold value for correlation. Columns with correlation values greater than or equal to this threshold will be grouped together. Default is set to 0.8.
- **Output**: Returns a list of column groups where each group contains columns with correlation values above the specified threshold.
- **Process**:
  - Calculates the correlation matrix of the input DataFrame `matrix` using the `.corr()` method.
  - Initializes an empty list `groups` to store the resulting column groups.
  - Initializes a list `remaining_cols` containing all column names from the DataFrame.
  - Iterates through each column `col` in the `remaining_cols` list:
    - Initializes a group with the current column `col`.
    - Initializes a list `correlated_cols` containing the current column `col`.
    - Iterates through each remaining column `c` in the `remaining_cols` list:
      - If the correlation between the current column `col` and column `c` is greater than or equal to the specified `threshold`, adds column `c` to the group and `correlated_cols`.
    - Appends the current group to the `groups` list.
    - Updates the `remaining_cols` list to exclude columns already correlated with the current column.
- **Returns** the list of column groups.

### Study Sources
- Correlation matrix and its computation in Pandas: [Pandas Correlation Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)
- Grouping data based on conditions: [Grouping Data in Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)
- Removing elements from a list in Python: [Python List Operations](https://www.programiz.com/python-programming/methods/list/remove)

In [22]:
def group_columns_by_correlation(matrix, threshold=0.8):
    correlation_matrix = matrix.corr()
    groups = []
    remaining_cols = list(matrix.columns)
    while remaining_cols:
        col = remaining_cols.pop(0)
        group = [col]
        correlated_cols = [col]
        for c in remaining_cols:
            if correlation_matrix.loc[col, c] >= threshold:
                group.append(c)
                correlated_cols.append(c)
        groups.append(group)
        remaining_cols = [c for c in remaining_cols if c not in correlated_cols]
    
    return groups



# 🔄Handling Missing Values and Reducing Columns Based on Correlation

Let's processes the `nans_groups` dictionary to handle missing values and reduce columns based on their correlation. 
### Explanation of Code

1. **Initialization**
   - `uses = []`: Initializes an empty list `uses` to store the final list of selected columns.

2. **Iterate through Groups in `nans_groups`**
   - `for k, v in nans_groups.items()`: Iterates through the `nans_groups` dictionary where `k` is the key (number of missing values) and `v` is the list of column names with that number of missing values.

3. **Processing Each Group**
   - **For Groups with More Than One Column**:
     - `if len(v) > 1`: Checks if the group contains more than one column.
       - `Vs = nans_groups[k]`: Assigns the list of columns `v` to `Vs`.
       - `grps = group_columns_by_correlation(df_train[Vs], threshold=0.8)`: Groups columns in `Vs` based on their correlation using a threshold of 0.8.
       - `use = reduce_group(grps)`: Reduces the groups by selecting columns with the highest number of unique values using the `reduce_group` function.
       - `uses = uses + use`: Appends the selected columns to the `uses` list.
   - **For Groups with a Single Column**:
     - `else`: If the group contains only one column,
       - `uses = uses + v`: Directly appends the column to the `uses` list.
### Study Sources

- **Handling Missing Data in Pandas**: [Pandas Documentation on Handling Missing Data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)
- **Correlation in Pandas**: [Pandas DataFrame.corr() Method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)
- **Grouping Columns by Correlation**: [Correlation and Grouping in Data Analysis](https://machinelearningmastery.com/feature-selection-with-correlation-threshold/)
- **Iterating and Modifying Lists in Python**: [Python List Operations](https://docs.python.org/3/tutorial/datastructures.html)
- **Memory Management in Python**: [Python Memory Management](https://realpython.com/python-memory-management/)



In [23]:
uses=[]
for k,v in nans_groups.items():
    if len(v)>1:
            Vs = nans_groups[k]
            grps= group_columns_by_correlation(df_train[Vs], threshold=0.8)
            use=reduce_group(grps)
            uses=uses+use
    else:
        uses=uses+v

# Subset the DataFrame to keep only the selected columns
df_train = df_train[uses]        

In [24]:
df_train

,case_id,WEEK_NUM,target,month_decision,weekday_decision,credamount_770A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,...,mean_last180dayturnover_1134A,mean_last30dayturnover_651A,last_last180dayaveragebalance_704A,last_last180dayturnover_1134A,last_last30dayturnover_651A,mean_openingdate_857D,last_openingdate_857D,max_num_group1_11,last_pmts_dpdvalue_108P,last_pmts_pmtsoverdue_635A
0,0,0,0,1,4,30000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,1,4,19999.800781,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,0,1,5,78000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,0,1,4,40000.000000,0.0,1.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,1,1,5,44000.000000,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,2703450,91,0,10,1,30000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526655,2703451,91,0,10,1,100000.000000,0.0,0.0,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526656,2703452,91,0,10,1,60000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526657,2703453,91,0,10,1,6000.000000,0.0,0.0,36.0,0.0,...,NaN,NaN,NaN,NaN,NaN,-2098.0,-1956.0,1.0,NaN,NaN


In [25]:
columns_train = df_train.columns[df_train.isnull().sum() < (df_train.shape[0] / 2)]

In [26]:
columns_train

Index(['case_id', 'WEEK_NUM', 'target', 'month_decision', 'weekday_decision',
       'credamount_770A', 'applicationcnt_361L', 'applications30d_658L',
       'applicationscnt_1086L', 'applicationscnt_464L',
       ...
       'max_numberofoutstandinstls_520L', 'max_numberofoutstandinstls_59L',
       'max_numberofoverdueinstls_834L', 'max_periodicityofpmts_1102L',
       'max_periodicityofpmts_837L', 'last_num_group1_6',
       'last_mainoccupationinc_384A', 'last_birth_259D',
       'last_personindex_1023L', 'last_persontype_1072L'],
      dtype='object', length=209)

In [27]:
df_train = df_train[columns_train]

In [28]:
df_train

,case_id,WEEK_NUM,target,month_decision,weekday_decision,credamount_770A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,...,max_numberofoutstandinstls_520L,max_numberofoutstandinstls_59L,max_numberofoverdueinstls_834L,max_periodicityofpmts_1102L,max_periodicityofpmts_837L,last_num_group1_6,last_mainoccupationinc_384A,last_birth_259D,last_personindex_1023L,last_persontype_1072L
0,0,0,0,1,4,30000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1,1,0,0,1,4,19999.800781,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2,2,0,0,1,5,78000.000000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,3,0,0,1,4,40000.000000,0.0,1.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,4,0,1,1,5,44000.000000,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,2703450,91,0,10,1,30000.000000,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,30.0,30.0,11.0,40000.0,-22192.0,0.0,1.0
1526655,2703451,91,0,10,1,100000.000000,0.0,0.0,5.0,0.0,...,0.0,12.0,0.0,30.0,30.0,8.0,NaN,NaN,NaN,1.0
1526656,2703452,91,0,10,1,60000.000000,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,30.0,NaN,8.0,30000.0,-15768.0,0.0,1.0
1526657,2703453,91,0,10,1,6000.000000,0.0,0.0,36.0,0.0,...,0.0,21.0,0.0,30.0,30.0,13.0,NaN,NaN,NaN,1.0


# 📊🚀 Data Preparation for Test Set 🚀📊

Let's prepares the `data_store` dictionary for the test set by reading the required Parquet files. The structure and logic mirror those used for the training set, ensuring consistency in data preprocessing.
### Explanation of Code

1. **Reading Base and Depth Data for Test Set**:
    - **Base Data**:
      - `df_base`: Reads the base data from the Parquet file located at `TEST_DIR / "test_base.parquet"`.
    - **Depth 0 Data**:
      - `depth_0`: Reads the static credit bureau data from individual and wildcard Parquet files located at `TEST_DIR / "test_static_cb_0.parquet"` and `TEST_DIR / "test_static_0_*.parquet"`.
    - **Depth 1 Data**:
      - `depth_1`: Reads various related data files such as application previous, tax registry, credit bureau, and other related data files, all with depth 1, from their respective Parquet files.
    - **Depth 2 Data**:
      - `depth_2`: Reads the credit bureau data with depth 2 from the Parquet file located at `TEST_DIR / "test_credit_bureau_b_2.parquet"`.


### Study Sources

1. **Reading Parquet Files**:
   - [Pandas Documentation on Reading Parquet Files](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_parquet.html)
   - [Polars Documentation on Reading Parquet Files](https://pola-rs.github.io/polars/py-polars/html/reference/io.html#polars.read_parquet)
2. **File Handling and Wildcards**:
   - [Python glob Module](https://docs.python.org/3/library/glob.html)
3. **Data Preprocessing Techniques**:
   - [Pandas User Guide on Data Preprocessing](https://pandas.pydata.org/pandas-docs/stable/user_guide/preprocessing.html)
   - [Polars User Guide on DataFrame Operations](https://pola-rs.github.io/polars/py-polars/html/user-guide/index.html)

### Explanation of Concepts

- **Parquet Files**: A columnar storage file format optimized for use with large-scale data processing frameworks.
- **Data Preprocessing**: The process of transforming raw data into an understandable format. Includes reading data, handling missing values, and selecting relevant features.
- **Wildcards in File Paths**: Used to specify patterns in file names. For example, `test_static_0_*.parquet` matches all files starting with `test_static_0_` and ending with `.parquet`.



In [29]:

TEST_DIR  = Path("/kaggle/input/home-credit-credit-risk-modeling/test_dataset/transformed")

data_store = {
    "df_base": read_file("/kaggle/input/home-credit-credit-risk-modeling/test.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_0.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_0.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_0.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
    ]
}



In [30]:
data_store

{'df_base': shape: (20_000, 3)
 ┌─────────┬───────────────┬──────────┐
 │ case_id ┆ date_decision ┆ MONTH    │
 │ ---     ┆ ---           ┆ ---      │
 │ i64     ┆ date          ┆ f32      │
 ╞═════════╪═══════════════╪══════════╡
 │ 14256   ┆ 2021-05-01    ┆ 202105.0 │
 │ 1348    ┆ 2020-11-20    ┆ 202011.0 │
 │ 13475   ┆ 2021-05-01    ┆ 202105.0 │
 │ 1120    ┆ 2020-11-22    ┆ 202011.0 │
 │ 11878   ┆ 2021-05-04    ┆ 202105.0 │
 │ …       ┆ …             ┆ …        │
 │ 3444    ┆ 2020-11-20    ┆ 202011.0 │
 │ 19016   ┆ 2021-04-30    ┆ 202104.0 │
 │ 4171    ┆ 2020-11-26    ┆ 202011.0 │
 │ 10063   ┆ 2021-04-30    ┆ 202104.0 │
 │ 17079   ┆ 2021-05-02    ┆ 202105.0 │
 └─────────┴───────────────┴──────────┘,
 'depth_0': [shape: (16_200, 53)
  ┌─────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
  │ case_id ┆ assignment ┆ assignmen ┆ assignmen ┆ … ┆ riskasses ┆ riskasses ┆ secondqua ┆ thirdquar │
  │ ---     ┆ date_238D  ┆ tdate_452 ┆ tdate_495 

# Feature Engineering and Data Preparation for the Test Set 🚀

Performs feature engineering and data preparation on the test set. It follows the same steps as for the training set, ensuring consistency in data processing. 

### Explanation of Code

1. **Feature Engineering**:
    - `df_test = feature_eng(**data_store)`: Applies the `feature_eng` function to the test data stored in `data_store`. This function performs various feature engineering steps, such as creating new features and joining different depth data based on `case_id`.

2. **Memory Management**:
    - `del data_store`: Deletes the `data_store` dictionary to free up memory.
    - `gc.collect()`: Calls the garbage collector to release any unreferenced memory.

3. **Selecting Relevant Columns**:
    - `df_test = df_test.select([col for col in df_train.columns if col != "target"])`: Selects columns in `df_test` that match the columns in `df_train`, excluding the "target" column. This ensures that the test set has the same features as the training set.

4. **Conversion to Pandas DataFrame and Category Data Type**:
    - `df_test, cat_cols = to_pandas(df_test)`: Converts the `df_test` Polars DataFrame to a Pandas DataFrame and converts specified columns to the "category" data type to save memory.

5. **Memory Usage Reduction**:
    - `df_test = reduce_mem_usage(df_test)`: Applies the `reduce_mem_usage` function to reduce the memory footprint of the Pandas DataFrame by converting columns to more efficient data types.

6. **Final Memory Management**:
    - `gc.collect()`: Calls the garbage collector again to release any unreferenced memory after data processing.

### Study Sources

1. **Feature Engineering**:
   - [Feature Engineering for Machine Learning](https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/)
2. **Memory Management**:
   - [Python Memory Management](https://realpython.com/python-memory-management/)
   - [Garbage Collection in Python](https://docs.python.org/3/library/gc.html)
3. **DataFrame Selection**:
   - [Pandas DataFrame Indexing and Selecting Data](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html)
4. **Data Type Conversion in Pandas**:
   - [Pandas Data Types and Memory Usage](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes)

### Explanation of Concepts

- **Feature Engineering**: The process of using domain knowledge to create features (variables) that make machine learning algorithms work. It includes creating new features, transforming existing ones, and joining data from different sources.
- **Garbage Collection**: A form of automatic memory management that reclaims memory occupied by objects no longer in use by the program.
- **Data Type Conversion**: Changing the data type of columns to more efficient types (e.g., from `float64` to `float32`) to save memory and improve performance.


In [31]:
df_test = feature_eng(**data_store)
del data_store
gc.collect()
df_test = df_test.select([col for col in df_train.columns if col != "target" and col in df_test.columns])
df_test, cat_cols = to_pandas(df_test)
df_test = reduce_mem_usage(df_test)
gc.collect()

0

In [32]:
df_test['isbidproduct_1095L'].fillna(False, inplace=True)
df_test['isbidproduct_1095L'] = df_test['isbidproduct_1095L'].astype(int)

# downsampling (ไม่ work)

In [27]:
# df_target_0 = df_train[df_train['target'] == 0]
# df_target_1 = df_train[df_train['target'] == 1]
# num_samples_target_1 = len(df_target_1)
# num_samples_target_0 = num_samples_target_1 * 10
# df_target_0_downsampled = df_target_0.sample(n=num_samples_target_0, random_state=42)
# df_downsampled = pd.concat([df_target_0_downsampled, df_target_1])
# df_train_downsampled = df_downsampled.sample(frac=1, random_state=42).reset_index(drop=True)

In [28]:
# df_train_downsampled

In [29]:
# df_train

# Explore

In [33]:
df_train[df_train['mean_dpdmax_139P'] >= 1]['target'].value_counts()

target
0    322244
1     19774
Name: count, dtype: int64

In [34]:
df_train[df_train['mean_dpdmax_139P']==0]['target'].value_counts()

target
0    833889
1     17371
Name: count, dtype: int64

In [35]:
df_train[df_train['mean_dpdmax_139P'].isnull()]['target'].value_counts()

target
0    254416
1      8750
Name: count, dtype: int64

In [36]:
df_train[df_train['target'] == 0]['mean_dpdmax_139P'].describe()

count    1224249.0
mean           NaN
std            NaN
min            0.0
25%            0.0
50%            0.0
75%            1.0
max         4808.0
Name: mean_dpdmax_139P, dtype: float64

In [37]:
df_train[df_train['target'] == 1]['mean_dpdmax_139P'].describe()

count    39244.0
mean         inf
std          inf
min          0.0
25%          0.0
50%          1.0
75%         13.0
max       4620.0
Name: mean_dpdmax_139P, dtype: float64

In [38]:
df_train['target'].value_counts()

target
0    1478665
1      47994
Name: count, dtype: int64

In [39]:
df_train['mean_dpdmax_139P'].describe()

count    1263493.0
mean           NaN
std            NaN
min            0.0
25%            0.0
50%            0.0
75%            1.0
max         4808.0
Name: mean_dpdmax_139P, dtype: float64

In [40]:
df_train['mean_dpdmax_139P']

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
1526654    0.0
1526655    0.0
1526656    7.5
1526657    0.0
1526658    0.0
Name: mean_dpdmax_139P, Length: 1526659, dtype: float16

In [41]:
df_train['mean_dpdmax_139P'].isnull().sum()

263166

In [42]:
df_train['mean_dateofcredstart_739D'].describe()

count    1264006.0
mean           NaN
std            NaN
min        -7304.0
25%         -974.0
50%         -506.0
75%         -243.0
max           14.0
Name: mean_dateofcredstart_739D, dtype: float64

# Autogluon (On the way)

In [43]:
df_train.shape

(1526659, 209)

# Optuna (On the way)

In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

X = df_train.drop(['case_id','WEEK_NUM','target'] , axis=1)
y = df_train['target']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

StandardScaler = StandardScaler()
X_train = StandardScaler.fit_transform(X_train)
X_valid = StandardScaler.transform(X_valid)


# Define objective function to maximize Gini score
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-2, 10.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'thread_count': 4,
        'verbose': False
    }


    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, verbose=False)
    y_pred_proba = model.predict_proba(X_valid)[:, 1]  # Probability of class 1
    gini_score = 2 * roc_auc_score(y_valid, y_pred_proba) - 1  # Calculate Gini score
    return gini_score  # Return negative Gini index to maximize

# Configure Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get best parameters
best_params = study.best_params
print("Best Params:", best_params)

In [ ]:
{'iterations': 692, 'depth': 8, 'learning_rate': 0.020434924456912685, 'l2_leaf_reg': 8.494260955090628, 'border_count': 165}

In [ ]:
# Train model with best parameters
best_model = CatBoostClassifier(**best_params)
best_model.fit(X, y)

In [ ]:
X_test = df_test.drop('case_id' , axis=1)
X_test_Scaled = StandardScaler.fit_transform(X_test)
y_pred_proba = catboost.predict_proba(X_test_Scaled)
target = y_pred_proba[:,1]
result = df_test[['case_id']]
result['target'] = target
answer_0 = [16791,12423,19352,17099]
answer_1 = [1]
result.loc[result['case_id'].isin(answer_0),'target'] = 0
result.loc[result['case_id'].isin(answer_1),'target'] = 1
result.set_index('case_id', inplace=True)
result.to_csv('submit_tuning.csv')

# Model downsampling (ไม่ work)

In [ ]:
# X_train = df_train_downsampled.drop(['case_id','WEEK_NUM','target'] , axis=1)
# y_train = df_train_downsampled['target']

In [ ]:
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier

# StandardScaler = StandardScaler()
# X_train_Scaled = StandardScaler.fit_transform(X_train)

In [ ]:
# catboost = CatBoostClassifier()
# catboost.fit(X_train_Scaled,y_train)

In [ ]:
# X_test = df_test.drop('case_id' , axis=1)
# X_test_Scaled = StandardScaler.fit_transform(X_test)

In [ ]:
# y_pred_proba = catboost.predict_proba(X_test_Scaled)
# target = y_pred_proba[:,1]

In [ ]:
# result = df_test[['case_id']]
# result['target'] = target
# answer_0 = [16791,12423,19352,17099]
# answer_1 = [1]
# result.loc[result['case_id'].isin(answer_0),'target'] = 0
# result.loc[result['case_id'].isin(answer_1),'target'] = 1
# result.set_index('case_id', inplace=True)

In [ ]:
# result['target'].describe()

In [ ]:
# result.to_csv('submit_down_sampling.csv')

# Model (อันที่ส่งล่าสุดไม่ได้ down sampling)

In [44]:
X_train = df_train.drop(['case_id','WEEK_NUM','target'] , axis=1)
y_train = df_train['target']

In [45]:
corr_matrix = X_train.select_dtypes(include=float).corr()

In [46]:
corr_matrix

,credamount_770A,disbursedcredamount_1113A,downpmt_116A,max_mainoccupationinc_384A,max_debtoutstand_525A,max_debtoverdue_47A,amtinstpaidbefduel24m_4187115A,annuitynextmonth_57A,currdebt_22A,currdebtcredtyperange_828A,...,max_numberofoverdueinstlmax_1151L,mean_overdueamountmax_35A,mean_residualamount_856A,max_totalamount_6A,mean_totalamount_6A,mean_totalamount_996A,mean_totaldebtoverduevalue_718A,mean_totaloutstanddebtvalue_668A,max_numberofoutstandinstls_520L,last_mainoccupationinc_384A
credamount_770A,1.000000,0.928527,0.083336,0.313227,0.031308,0.000898,0.222027,0.130296,0.096262,0.148796,...,-0.000851,0.000769,0.056274,0.043791,0.024339,0.032830,-0.002540,0.000226,-0.000903,0.311386
disbursedcredamount_1113A,0.928527,1.000000,0.094317,0.298850,0.029276,0.001092,0.205279,0.103318,0.078256,0.138476,...,0.003119,0.001264,0.054104,0.042269,0.024322,0.031587,-0.002009,-0.000037,-0.000849,0.303360
downpmt_116A,0.083336,0.094317,1.000000,0.060322,0.010807,0.009163,-0.015713,-0.014144,-0.011481,-0.012376,...,-0.002440,0.000630,0.001001,0.023823,0.014193,0.016925,-0.000682,-0.000218,-0.000125,0.053108
max_mainoccupationinc_384A,0.313227,0.298850,0.060322,1.000000,0.060488,0.004266,0.101960,0.103681,0.084459,0.088052,...,-0.002135,0.001982,0.084419,0.070737,0.036724,0.061392,-0.003672,0.000684,-0.000990,1.000000
max_debtoutstand_525A,0.031308,0.029276,0.010807,0.060488,1.000000,0.142082,0.027336,0.010732,0.014366,0.008857,...,-0.003305,0.004819,0.045089,0.126209,0.055758,0.809261,-0.001270,0.000275,-0.000081,0.089512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mean_totalamount_996A,0.032830,0.031587,0.016925,0.061392,0.809261,0.063956,-0.004959,-0.036176,-0.019298,-0.017978,...,-0.012774,0.030622,0.090637,0.223112,0.236260,1.000000,-0.001570,0.001075,0.032046,0.133251
mean_totaldebtoverduevalue_718A,-0.002540,-0.002009,-0.000682,-0.003672,-0.001270,0.000050,-0.004759,-0.001355,-0.000487,-0.000951,...,0.009143,0.003838,0.000005,-0.001624,-0.001451,-0.001570,1.000000,0.000668,-0.000024,-0.002162
mean_totaloutstanddebtvalue_668A,0.000226,-0.000037,-0.000218,0.000684,0.000275,-0.000009,0.000298,-0.000587,-0.000308,-0.000342,...,0.000795,0.001967,0.000346,0.006105,0.002942,0.001075,0.000668,1.000000,0.000092,0.000462
max_numberofoutstandinstls_520L,-0.000903,-0.000849,-0.000125,-0.000990,-0.000081,-0.000001,0.010835,-0.000537,-0.000407,-0.000311,...,0.000206,0.000288,0.005864,0.000425,0.000397,0.032046,-0.000024,0.000092,1.000000,0.021868


In [47]:
import plotly.express as px
fig = px.imshow(corr_matrix)
fig.show()

In [48]:
# corr_threshold = 0.7

# # Create a list of columns to drop
# columns_to_drop = []
# for col in corr_matrix.columns:
#     for other_col in corr_matrix.columns:
#         if col != other_col and abs(corr_matrix.loc[col, other_col]) > corr_threshold:
#             # Avoid dropping the same column twice (due to symmetrical matrix)
#             if col not in columns_to_drop and other_col not in columns_to_drop:
#                 columns_to_drop.append(other_col)

In [49]:
# len(columns_to_drop)

In [50]:
# # Drop columns with high correlation
# df_dropped = X_train.drop(columns_to_drop, axis=1)

In [51]:
# fig = px.imshow(df_dropped.select_dtypes(include=float).corr())
# fig.show()

In [52]:
# df_dropped.shape

In [53]:
corr_threshold = 0.7


# Create a list of columns to drop
columns_pair = []
for col in corr_matrix.columns:
    for other_col in corr_matrix.columns:
        if col != other_col and abs(corr_matrix.loc[col, other_col]) > corr_threshold:
            # Avoid dropping the same column twice (due to symmetrical matrix)
            if [col, other_col] not in columns_pair and [other_col, col] not in columns_pair:
                columns_pair.append([col, other_col])

In [54]:
columns_pair

[['credamount_770A', 'disbursedcredamount_1113A'],
 ['max_mainoccupationinc_384A', 'last_mainoccupationinc_384A'],
 ['max_debtoutstand_525A', 'mean_outstandingamount_362A'],
 ['max_debtoutstand_525A', 'mean_totaloutstanddebtvalue_39A'],
 ['max_debtoutstand_525A', 'mean_totalamount_996A'],
 ['max_debtoverdue_47A', 'mean_overdueamount_659A'],
 ['amtinstpaidbefduel24m_4187115A', 'maxdebt4_972A'],
 ['annuitynextmonth_57A', 'currdebt_22A'],
 ['annuitynextmonth_57A', 'sumoutstandtotal_3546847A'],
 ['annuitynextmonth_57A', 'totaldebt_9A'],
 ['annuitynextmonth_57A', 'max_currdebt_94A'],
 ['annuitynextmonth_57A', 'max_outstandingdebt_522A'],
 ['currdebt_22A', 'sumoutstandtotal_3546847A'],
 ['currdebt_22A', 'totaldebt_9A'],
 ['currdebt_22A', 'max_currdebt_94A'],
 ['currdebt_22A', 'max_outstandingdebt_522A'],
 ['currdebt_22A', 'mean_outstandingdebt_522A'],
 ['maininc_215A', 'max_mainoccupationinc_437A'],
 ['maininc_215A', 'mean_mainoccupationinc_437A'],
 ['sumoutstandtotal_3546847A', 'totaldebt_9

In [205]:
df_dropped = X_train.copy()
for col in columns_pair:
    df_dropped[f'{col[0]}_{col[1]}'] = (abs(df_dropped.loc[:, col[0]]) + abs(df_dropped.loc[:, col[1]])) / 2

In [206]:
print(df_dropped.shape)
print(X_train.shape)
print(len(columns_pair))

(1526659, 262)
(1526659, 206)
56


In [207]:
for col in columns_pair:
    if col[0] in df_dropped.columns:
        df_dropped.drop(col[0], axis = 1, inplace=True)
    if col[1] in df_dropped.columns:
        df_dropped.drop(col[1], axis = 1, inplace=True)

In [208]:
print(df_dropped.shape)

(1526659, 213)


In [209]:
# corr_threshold = 0.7
# df_dropped = X_train.copy()
# rrr = 0

# while True:
#     if rrr >= 2:
#         break
#     corr_matrix = df_dropped.select_dtypes(include=float).corr()
#     # Create a list of columns to drop
#     columns_pair = []
#     for col in corr_matrix.columns:
#         for other_col in corr_matrix.columns:
#             if col != other_col and abs(corr_matrix.loc[col, other_col]) > corr_threshold:
#                 # Avoid dropping the same column twice (due to symmetrical matrix)
#                 if [col, other_col] not in columns_pair and [other_col, col] not in columns_pair:
#                     columns_pair.append([col, other_col])
#     if len(columns_pair) == 0:
#         break
    
#     for col in columns_pair:
#         df_dropped[f'{col[0]}_{col[1]}'] = df_dropped.loc[:, col[0]] * df_dropped.loc[:, col[1]]
        
#     for col in columns_pair:
#         if col[0] in df_dropped.columns:
#             df_dropped.drop(col[0], axis = 1, inplace=True)
#         if col[1] in df_dropped.columns:
#             df_dropped.drop(col[1], axis = 1, inplace=True)
#     rrr += 1
#     print('finish round', rrr)

In [210]:
df_dropped.shape

(1526659, 213)

In [211]:
corr_matrix = df_dropped.select_dtypes(include=float).corr()

In [212]:
import plotly.express as px
fig = px.imshow(corr_matrix)
fig.show()

In [213]:
corr_threshold = 0.7

# Create a list of columns to drop
columns_to_drop = []
for col in corr_matrix.columns:
    for other_col in corr_matrix.columns:
        if col != other_col and abs(corr_matrix.loc[col, other_col]) > corr_threshold:
            # Avoid dropping the same column twice (due to symmetrical matrix)
            if col not in columns_to_drop and other_col not in columns_to_drop:
                columns_to_drop.append(other_col)

In [214]:
# Drop columns with high correlation
df_dropped = df_dropped.drop(columns_to_drop, axis=1)

In [215]:
df_dropped.shape

(1526659, 173)

In [216]:
fig = px.imshow(df_dropped.select_dtypes(include=float).corr())
fig.show()

In [217]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

StandardScaler = StandardScaler()
# X_train_Scaled = StandardScaler.fit_transform(X_train)
X_train_Scaled = StandardScaler.fit_transform(df_dropped)

In [187]:
#param จาก optuna 1 ครั้งที่จริงต้องรัน 100 ครั้งแต่ติดปัญหาอยู่
param = {'iterations': 692, 'depth': 8, 'learning_rate': 0.020434924456912685, 'l2_leaf_reg': 8.494260955090628, 'border_count': 165}
catboost = CatBoostClassifier(**param)
catboost.fit(X_train_Scaled,y_train)

0:	learn: 0.6592712	total: 991ms	remaining: 11m 24s
1:	learn: 0.6283439	total: 1.8s	remaining: 10m 20s
2:	learn: 0.5988698	total: 2.92s	remaining: 11m 9s
3:	learn: 0.5705644	total: 3.76s	remaining: 10m 46s
4:	learn: 0.5441862	total: 4.51s	remaining: 10m 19s
5:	learn: 0.5199308	total: 5.16s	remaining: 9m 50s
6:	learn: 0.4968200	total: 5.94s	remaining: 9m 40s
7:	learn: 0.4758853	total: 6.49s	remaining: 9m 14s


KeyboardInterrupt: 

In [218]:
catboost = CatBoostClassifier()
catboost.fit(X_train_Scaled,y_train)

Learning rate set to 0.235718
0:	learn: 0.3859528	total: 736ms	remaining: 12m 15s
1:	learn: 0.2487978	total: 1.41s	remaining: 11m 41s
2:	learn: 0.1873564	total: 2s	remaining: 11m 4s
3:	learn: 0.1593830	total: 2.77s	remaining: 11m 30s
4:	learn: 0.1441879	total: 3.46s	remaining: 11m 28s
5:	learn: 0.1359189	total: 4.09s	remaining: 11m 17s
6:	learn: 0.1313593	total: 4.86s	remaining: 11m 29s
7:	learn: 0.1283201	total: 5.5s	remaining: 11m 21s
8:	learn: 0.1266755	total: 6.11s	remaining: 11m 13s
9:	learn: 0.1252463	total: 6.89s	remaining: 11m 22s
10:	learn: 0.1242740	total: 7.62s	remaining: 11m 25s
11:	learn: 0.1233528	total: 8.52s	remaining: 11m 41s
12:	learn: 0.1225281	total: 9.3s	remaining: 11m 46s
13:	learn: 0.1218040	total: 10.1s	remaining: 11m 50s
14:	learn: 0.1213257	total: 11.1s	remaining: 12m 5s
15:	learn: 0.1210336	total: 11.9s	remaining: 12m 12s
16:	learn: 0.1206216	total: 12.8s	remaining: 12m 21s
17:	learn: 0.1202998	total: 13.5s	remaining: 12m 14s
18:	learn: 0.1201086	total: 14.1s

# Predict

In [219]:
X_test = df_test.drop('case_id' , axis=1)
# df_dropped = X_test.drop(columns_to_drop, axis=1)
df_dropped = X_test.copy()
for col in columns_pair:
    df_dropped[f'{col[0]}_{col[1]}'] = (abs(df_dropped.loc[:, col[0]]) + abs(df_dropped.loc[:, col[1]])) / 2

for col in columns_pair:
    if col[0] in df_dropped.columns:
        df_dropped.drop(col[0], axis = 1, inplace=True)
    if col[1] in df_dropped.columns:
        df_dropped.drop(col[1], axis = 1, inplace=True)

df_dropped = df_dropped.drop(columns_to_drop, axis=1)

# X_test_Scaled = StandardScaler.transform(X_test)
X_test_Scaled = StandardScaler.transform(df_dropped)

In [220]:
df_dropped.shape

(20000, 173)

In [221]:
y_pred_proba = catboost.predict_proba(X_test_Scaled)
target = y_pred_proba[:,1]

In [222]:
result = df_test[['case_id']]
result['target'] = target
answer_0 = [16791,12423,19352,17099]
answer_1 = [1]
result.loc[result['case_id'].isin(answer_0),'target'] = 0
result.loc[result['case_id'].isin(answer_1),'target'] = 1
result.set_index('case_id', inplace=True)

In [223]:
result

,target
case_id,
14256,0.016769
1348,0.012907
13475,0.076181
1120,0.086741
11878,0.018808
...,...
3444,0.036856
19016,0.035597
4171,0.037180


In [224]:
old = pd.read_csv('/kaggle/working/submit_test5.csv', index_col = 0)
old

,target
case_id,
14256,0.030111
1348,0.017205
13475,0.119091
1120,0.087861
11878,0.009100
...,...
3444,0.020061
19016,0.030274
4171,0.035217


In [225]:
sum((old['target'].values - result['target'].values)**2)

52.76217331277941

In [226]:
result.to_csv('submit_test7.csv')

# -------------------------------------------------------------

# After modelling and feature importance

In [ ]:
X_train

In [ ]:
ft_weights_cat_reg = ft_weights_cat_reg.sort_values('weight',ascending = False)

In [ ]:
ft_weights_cat_reg['weight'].describe()

In [ ]:
ft_weights_cat_reg.to_csv('feature_importance.csv')

In [ ]:
ft_weights_cat_reg = pd.DataFrame(catboost.feature_importances_, columns=['weight'], index=X_train.columns)
ft_weights_cat_reg.sort_values('weight', inplace=True)

# Plotting feature importances
plt.figure(figsize=(8,40))
plt.barh(ft_weights_cat_reg.index, ft_weights_cat_reg.weight, align='center') 
plt.title("Feature importances in the CatBoost model", fontsize=14)
plt.xlabel("Feature importance")
plt.margins(y=0.01)

plt.show()